### Github Config

In [118]:
!mkdir -p ~/.ssh
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!chmod 600 ~/.ssh/known_hosts

# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16


In [119]:
!git clone git@github.com:ivanckng/CS610_AML.git

Cloning into 'CS610_AML'...
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [120]:
!ls -al ~/.ssh

total 16
drwxr-xr-x 2 root root 4096 May 10 08:49 .
drwx------ 1 root root 4096 May 10 08:49 ..
-rw------- 1 root root  828 May 10 08:49 known_hosts


In [121]:
!ssh-keygen -t rsa -b 4096 -C "ivanjunqiwu99@gmail.com"

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:Zb6bT+xIULiTbU6GUl9UReoDf8FsTRoOcwOguUxxmGs ivanjunqiwu99@gmail.com
The key's randomart image is:
+---[RSA 4096]----+
|        .oo.=o=o+|
|        o* . =o*.|
|        *.+ o +=.|
|       +E% . +. .|
|      ..S B   + .|
|       . B o   o |
|          + o    |
|         . *     |
|          +.o    |
+----[SHA256]-----+


In [122]:
!cat ~/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCKopS+5gHoNkhiIKjuk/hE4V3M5UxhjzPVn/enXu+5qz8qqjRnWzZvNFiLRdc8h4aGtdxY7zWvedEaIp6pwHTvqrPYgTuJtcvdyoADRgCBJxJxUOLQUBJTRCzLoiS2dr8InA6DUWWxQcGeUUCbNc1z121cGU69LO488X2M9CYe+sRY4wGN6pNYy2FhO5cD55hIAYXi997/XmJXTLJ4EJ6wjxyhhg6mbp0x4YaPvsUIhXt8PIN/OZDQRzuNoBeEpEJf+8a4/nblTyvWuVTmU4az1AlX7dQTFf2MN6q3MsmppMzX/ZQEvY7JC9R9VPQ9NAVNq9JQdulCyYxgZiqd9mHkPRf8xo0fdU6G+7AbquVBcFcaCGeFjjwxCT3yjZNfilCAh3eoF29nhKadJdK7foi+MXdachpNMBY9DrNOzugUvDPbpNiwXwaDRpmAcQMKTxoSqaXGNgPFxBtSgu2HDDdm4PvjH20emQLhwiCxoVdffQtGjwPAGpzY1w/o91MDqrrg7JjVsKfli2VYiJQfwlhx+MVsVEsI6HvdX+r/sMUqIfvNIsQERh+ElKhxMi9Pz3DYBvqJVcIgAmbZsypLQif0J4ZJpGVD+1O5S2M9JWfVnh/s0k10rbok/c2mOH24rYt6Tm/+sPjLf0Ge6x/WU5nWD01rh9F22qXkjiGRaultsw== ivanjunqiwu99@gmail.com


In [124]:
!git clone git@github.com:ivanckng/CS610_AML.git

Cloning into 'CS610_AML'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 5), reused 17 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 4.86 MiB | 3.04 MiB/s, done.
Resolving deltas: 100% (5/5), done.


### Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xgboost

In [3]:
!pip install nltk

In [84]:
import pandas as pd
from sklearn import linear_model, metrics, preprocessing, tree, model_selection, feature_selection, ensemble
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from collections import Counter

### Loading training and tesing data

In [206]:
train_set = pd.read_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/test.csv')

### Data Processing & EDA

In [207]:
train_set.head()

,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,lang,location,profile_background_image_url,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days,target
0,2012-01-15 23:40:09,True,False,Cosplayer/Fitness lover. Come to me https://t....,74,7,0,False,465096524,en,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9666745212...,reml5477,20,False,0.006,3138,1
1,2016-10-04 00:44:39,False,False,pobody’s nerfect,50443,164,590,True,783105517673648132,cy,she/her,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1281752126...,kinlibra,6469,False,4.572,1415,0
2,2009-05-23 04:04:13,False,False,gracias por participar 🏅,9394,208,189,False,41970759,es,La diaspora,http://abs.twimg.com/images/themes/theme17/bg.gif,http://pbs.twimg.com/profile_images/1233811596...,_delaualau,30296,False,7.378,4106,0
3,2009-05-17 04:31:31,False,False,Stand Up Comedian/Actor from North Philadelphi...,46,66180,1090,True,40607946,en,"Calabasas, CA",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1184851104...,SpankHorton,164957,False,40.116,4112,0
4,2009-02-16 13:11:21,True,False,Assignment Editor at NBC10 and President of Ja...,1223,487,867,True,20983433,en,"Jenkintown, PA",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/5234863934...,javelinjt,1752,False,0.417,4201,0


In [208]:
test_set.head()

,index,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,lang,location,profile_background_image_url,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days
0,0,2007-03-27 22:56:04,False,False,Dad. #ADHD @fasternormal Pod host. Author/keyn...,22290,163813,1746,True,2569261,en,"New York, NY",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1186468889...,petershankman,42013,True,8.586,4893
1,1,2013-03-30 08:47:01,True,False,NaN,685,44,637,False,1316038452,NaN,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9637876928...,Mihail_HD,111,False,0.041,2699
2,2,2009-04-25 15:24:08,False,False,#UGAMOM #GODAWGS,12257,276,194,True,35233977,de,"Augusta, GA",http://abs.twimg.com/images/themes/theme3/bg.gif,http://pbs.twimg.com/profile_images/1246470377...,TinkerMazell,18142,False,4.390,4133
3,3,2010-08-06 22:21:56,True,False,"golfer, traveler, car guy, PCA member, owns 19...",30806,1124,4999,True,175542625,en,New York,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1391357162...,joecangolfICC,20624,False,5.627,3665
4,4,2015-03-02 12:29:39,True,False,Do you like it gently? Come in! 💚💚💚 https://t....,224,22,265,False,3056068536,en,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9680856906...,TiesiogDovydas,64,False,0.032,1996


In [209]:
print("Size of Training Instances:", train_set.shape[0])
print("Sice of Features:", train_set.shape[1])
print(train_set.info())

print("Size of Testing Instances:", test_set.shape[0])
print("Sice of Features:", test_set.shape[1])
print(test_set.info())

Size of Training Instances: 26206
Sice of Features: 19
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26206 entries, 0 to 26205
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   created_at                    26206 non-null  object 
 1   default_profile               26206 non-null  bool   
 2   default_profile_image         26206 non-null  bool   
 3   description                   21115 non-null  object 
 4   favourites_count              26206 non-null  int64  
 5   followers_count               26206 non-null  int64  
 6   friends_count                 26206 non-null  int64  
 7   geo_enabled                   26206 non-null  bool   
 8   id                            26206 non-null  int64  
 9   lang                          20618 non-null  object 
 10  location                      26204 non-null  object 
 11  profile_background_image_url  22971 non-null  object 
 12  profi

In [210]:
train_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name', 'location'], inplace=True)
test_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name', 'location'], inplace=True)

In [211]:
train_set['description'] = train_set['description'].isna().astype(int)
train_set['description'] = np.where(train_set['description'] == 1, 0, 1)

# train_set['lang'] = train_set['lang'].isna().astype(int)
# train_set['lang'] = np.where(train_set['lang'] == 1, 0, 1)

# train_set['location'] = train_set['location'].isna().astype(int)
# train_set['location'] = np.where(train_set['location'] == 1, 0, 1)

test_set['description'] = test_set['description'].isna().astype(int)
test_set['description'] = np.where(test_set['description'] == 1, 0, 1)

# test_set['lang'] = test_set['lang'].isna().astype(int)
# test_set['lang'] = np.where(test_set['lang'] == 1, 0, 1)

# test_set['location'] = test_set['location'].isna().astype(int)
# test_set['location'] = np.where(test_set['location'] == 1, 0, 1)

In [212]:
train_set['lang'] = train_set['lang'].fillna('unknown').astype('category')
test_set['lang'] = test_set['lang'].fillna('unknown').astype('category')

In [213]:
train_set['default_profile'] = train_set['default_profile'].astype(int)
train_set['default_profile_image'] = train_set['default_profile_image'].astype(int)
train_set['geo_enabled'] = train_set['geo_enabled'].astype(int)
train_set['verified'] = train_set['verified'].astype(int)

test_set['default_profile'] = test_set['default_profile'].astype(int)
test_set['default_profile_image'] = test_set['default_profile_image'].astype(int)
test_set['geo_enabled'] = test_set['geo_enabled'].astype(int)
test_set['verified'] = test_set['verified'].astype(int)

In [214]:
num_majority = train_set[train_set['target'] == 0].shape[0]
num_minority = train_set[train_set['target'] == 1].shape[0]

In [215]:
# x_train = train_set.drop(columns=['target'])
# y_train = train_set['target']

x_train, x_test, y_train, y_test = model_selection.train_test_split(train_set.drop(columns=['target']), train_set['target'], test_size=0.2, shuffle = True, random_state=42, stratify=train_set['target'])
#

In [216]:
counter = Counter(y_train)
estimate_scale_pos_weight = counter[0] / counter[1]

In [217]:
train_predictors = train_set.drop(columns=['target'])
train_target = train_set['target']

In [218]:
counter_train = Counter(train_target)
estimate_scale_pos_weight_train = counter_train[0] / counter_train[1]

In [219]:
x_train

,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,lang,statuses_count,verified,average_tweets_per_day,account_age_days
16919,0,0,1,1534,45,222,0,pt,2759,0,0.705,3915
3662,0,0,1,2207,11557,820,1,en,7577,0,1.984,3820
6669,0,0,1,738,36684,185,1,ar,141155,0,42.414,3328
7412,0,0,1,24553,970,968,1,en,10305,0,2.783,3703
8506,0,0,1,8942,1898,807,1,en,10848,1,2.601,4170
...,...,...,...,...,...,...,...,...,...,...,...,...
17356,0,0,1,2768,295,295,1,en,13869,0,4.086,3394
92,0,0,1,63616,883,742,1,it,52246,0,25.954,2013
8298,0,0,1,3256,341,186,1,en,1580,0,0.386,4092
14834,1,0,1,1332,8,31,0,en,1333,0,0.912,1462


### Logistic Regression

In [ ]:
logreg_cv_model = linear_model.LogisticRegressionCV(
    Cs=10,
    cv=10,
    penalty='l1',
    solver='liblinear',
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1,
    max_iter=1000
)

In [ ]:
logreg_cv_model.fit(x_train, y_train)

LogisticRegressionCV(cv=10, max_iter=1000, n_jobs=-1, penalty='l1',
                     random_state=42, scoring='roc_auc', solver='liblinear')

In [ ]:
param_grid_logreg = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}
grid_search_logreg = model_selection.GridSearchCV(
    linear_model.LogisticRegression(random_state=42, max_iter=1000),
    param_grid_logreg,
    cv=10,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

In [ ]:
grid_search_logreg.fit(x_train, y_train)
best_logreg = grid_search_logreg.best_estimator_

Fitting 10 folds for each of 12 candidates, totalling 120 fits


In [ ]:
y_train_pred_proba = logreg_cv_model.predict_proba(x_train)[:, 1]
train_auc = metrics.roc_auc_score(y_train, y_train_pred_proba)
print("Train AUC:", train_auc)
y_test_pred_proba = logreg_cv_model.predict_proba(x_test)[:, 1]
test_auc = metrics.roc_auc_score(y_test, y_test_pred_proba)
print("Test AUC:", test_auc)

Train AUC: 0.8254802670676067
Test AUC: 0.8289852828555255


### Decision Tree

In [ ]:
dt_model = tree.DecisionTreeClassifier(random_state=42)

In [ ]:
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20,30,40],
    'min_samples_split': [2, 3, 5, 7, 10, 15],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': [None, 'sqrt', 'log2']
}

In [ ]:
grid_search_dt = model_selection.GridSearchCV(estimator=dt_model,
                              param_grid=param_grid_dt,
                              cv=5,
                              scoring='roc_auc',
                              n_jobs=-1,
                              verbose=1
                             )

In [ ]:
grid_search_dt.fit(x_train, y_train)

Fitting 5 folds for each of 1260 candidates, totalling 6300 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 15, 20, 30, 40],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 3, 5, 7, 10, 15]},
             scoring='roc_auc', verbose=1)

In [ ]:
best_dt_model = grid_search_dt.best_estimator_

In [ ]:
y_train_pred_proba_dt = best_dt_model.predict_proba(x_train)[:, 1]
train_auc_dt = metrics.roc_auc_score(y_train, y_train_pred_proba_dt)
print(f"Train AUC for Decision Tree: {train_auc_dt:.4f}")
y_test_pred_proba_dt = best_dt_model.predict_proba(x_test)[:, 1]
test_auc_dt = metrics.roc_auc_score(y_test, y_test_pred_proba_dt)
print(f"Test AUC for Decision Tree: {test_auc_dt:.4f}")

Train AUC for Decision Tree: 0.9515
Test AUC for Decision Tree: 0.9050


In [ ]:
feature_importances = best_dt_model.feature_importances_
feature_importances_df = pd.DataFrame({'feature': x_train.columns, 'importance': feature_importances})
print(feature_importances_df.sort_values(by='importance', ascending=False).head(10))

                   feature  importance
4          followers_count    0.403168
3         favourites_count    0.187233
11  average_tweets_per_day    0.102436
12        account_age_days    0.083662
5            friends_count    0.074960
9           statuses_count    0.069607
10                verified    0.056756
6              geo_enabled    0.011844
2              description    0.004410
0          default_profile    0.003158


In [ ]:
x_train_selected = x_train.loc[:, feature_importances > 0.05]
x_test_selected = x_test.loc[:, feature_importances > 0.05]

In [ ]:
grid_search_dt.fit(x_train_selected, y_train)

Fitting 5 folds for each of 1260 candidates, totalling 6300 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 15, 20, 30, 40],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 3, 5, 7, 10, 15]},
             scoring='roc_auc', verbose=1)

In [ ]:
best_dt_model = grid_search_dt.best_estimator_

In [ ]:
y_train_pred_proba_dt = best_dt_model.predict_proba(x_train_selected)[:, 1]
train_auc_dt = metrics.roc_auc_score(y_train, y_train_pred_proba_dt)
print(f"Train AUC for Decision Tree: {train_auc_dt:.4f}")
y_test_pred_proba_dt = best_dt_model.predict_proba(x_test_selected)[:, 1]
test_auc_dt = metrics.roc_auc_score(y_test, y_test_pred_proba_dt)
print(f"Test AUC for Decision Tree: {test_auc_dt:.4f}")

Train AUC for Decision Tree: 0.9476
Test AUC for Decision Tree: 0.9038


In [ ]:
y_test_pred_dt = best_dt_model.predict(test_set.iloc[:,1:])
pred_results_dt = pd.DataFrame({'index': test_set['index'], 'target': y_test_pred_dt})
pred_results_dt.to_csv('pred_results_dt.csv', index=False)

### Random Forest

In [131]:
rf_model = ensemble.RandomForestClassifier(
    random_state=42,
    class_weight='balanced_subsample',
)

In [132]:
param_grid_rf = {
    'n_estimators': [100, 200, 300, 400, 500],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}

In [133]:
kfold = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [134]:
grid_search_rf = model_selection.GridSearchCV(estimator=rf_model,
                              param_grid=param_grid_rf,
                              cv=kfold,
                              scoring='roc_auc',
                              n_jobs=-1,
                              verbose=2
                             )

In [135]:
grid_search_rf.fit(x_train, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=2)

In [137]:
best_rf_model = grid_search_rf.best_estimator_

In [138]:
y_train_pred_proba_rf = best_rf_model.predict_proba(x_train)[:, 1]
train_auc_rf = metrics.roc_auc_score(y_train, y_train_pred_proba_rf)
print(f"Train AUC for tuned Random Forest: {train_auc_rf:.4f}")
y_test_pred_proba_rf = best_rf_model.predict_proba(x_test)[:, 1]
test_auc_rf = metrics.roc_auc_score(y_test, y_test_pred_proba_rf)
print(f"Test AUC for tuned Random Forest: {test_auc_rf:.4f}")

Train AUC for tuned Random Forest: 1.0000
Test AUC for tuned Random Forest: 0.9408


In [ ]:
train_predictors = train_set.drop(columns=['target'])
train_target = train_set['target']

In [ ]:
grid_search_rf.fit(train_predictors, train_target)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=2)

In [ ]:
best_rf_model = grid_search_rf.best_estimator_

In [ ]:
train_target_pred_proba_rf = best_rf_model.predict_proba(train_predictors)[:, 1]
train_auc_rf = metrics.roc_auc_score(train_target, train_target_pred_proba_rf)
print(f"Train AUC for tuned Random Forest: {train_auc_rf:.4f}")

Train AUC for tuned Random Forest: 1.0000


In [ ]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_rf_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('rf_test_set_prob.csv', index=False)

### XGBoost

In [224]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False,
eval_metric='logloss',
random_state=42,
tree_method='hist',
device='cuda',
scale_pos_weight=estimate_scale_pos_weight,
enable_categorical = True
)  # Important:  Set eval_metric

In [225]:
param_grid_xgb = {
    'n_estimators': [100, 200,  300,  400, 500],
    'learning_rate': [0.001,0.01, 0.02, 0.05],
}

In [226]:
grid_search_xgb = model_selection.GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='roc_auc',
    cv=10,
    n_jobs=-1,
    verbose=1
)

In [227]:
grid_search_xgb.fit(x_train, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:05:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rat...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02, 0.05],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=1)

In [228]:
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_train_proba_xgb = best_xgb_model.predict_proba(x_train)[:, 1]
train_auc_xgb = metrics.roc_auc_score(y_train, y_pred_train_proba_xgb)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")
y_pred_test_proba_xgb = best_xgb_model.predict_proba(x_test)[:, 1]
test_auc_xgb = metrics.roc_auc_score(y_test, y_pred_test_proba_xgb)
print(f"Test AUC for tuned XGBoost: {test_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9675
Test AUC for tuned XGBoost: 0.9425


In [229]:
feature_importances = best_xgb_model.feature_importances_
feature_importances_df = pd.DataFrame({'feature': x_train.columns, 'importance': feature_importances})
print(feature_importances_df.sort_values(by='importance', ascending=False))

                   feature  importance
9                 verified    0.437337
4          followers_count    0.178024
6              geo_enabled    0.074462
3         favourites_count    0.055930
7                     lang    0.046332
5            friends_count    0.045689
10  average_tweets_per_day    0.045410
11        account_age_days    0.037923
8           statuses_count    0.036130
0          default_profile    0.030361
2              description    0.006897
1    default_profile_image    0.005506


In [161]:
grid_search_xgb.fit(x_train_selected, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:39:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_ra...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02, 0.05],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=1)

In [162]:
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_train_proba_xgb = best_xgb_model.predict_proba(x_train_selected)[:, 1]
train_auc_xgb = metrics.roc_auc_score(y_train, y_pred_train_proba_xgb)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")
y_pred_test_proba_xgb = best_xgb_model.predict_proba(x_test_selected)[:, 1]
test_auc_xgb = metrics.roc_auc_score(y_test, y_pred_test_proba_xgb)
print(f"Test AUC for tuned XGBoost: {test_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9727
Test AUC for tuned XGBoost: 0.9412


In [230]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False,
eval_metric='logloss',
random_state=42,
tree_method='hist',
device='cuda',
scale_pos_weight=estimate_scale_pos_weight_train
)  # Important:  Set eval_metric

In [231]:
# Original XGBoost model
grid_search_xgb.fit(train_predictors, train_target)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:12:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rat...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02, 0.05],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=1)

In [232]:
best_xgb_model = grid_search_xgb.best_estimator_
train_target_proba = best_xgb_model.predict_proba(train_predictors)[:, 1]
train_auc_xgb = metrics.roc_auc_score(train_target, train_target_proba)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9821


In [235]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_xgb_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('/content/drive/MyDrive/10 Github/CS610_AML/xgb_test_set_prob.csv', index=False)